# Extract Event Log Prefixes

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')
import sys
import os
#PATH = '/content/drive/My Drive/PPM_Stability/'
PATH = os.getcwd()
sys.path.append(PATH)

In [2]:
from DatasetManager import DatasetManager
import pickle

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
dataset_ref = "bpic2012"
gap = 1

dataset_ref_to_datasets = {
    "bpic2012" : ["bpic2012_accepted"],
    "sepsis_cases": ["sepsis_cases_1"],# "sepsis_cases_2", "sepsis_cases_4"]
    "production" : ["production"],
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]

train_ratio = 0.8
random_state = 22

print(datasets)

['bpic2011_f1']


In [4]:
for dataset_name in datasets:
    
    # read the data
    print("setting up data...")
    dataset_manager = DatasetManager(dataset_name)
    data = dataset_manager.read_dataset()
    cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

    # determine min and max (truncated) prefix lengths
    min_prefix_length = 1
    if "traffic_fines" in dataset_name:
        max_prefix_length = 10
    elif "bpic2017" in dataset_name:
        max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    else:
        max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))

    # split into training and test
    train, test = dataset_manager.split_data_strict(data, train_ratio, split="temporal")
    train, val = dataset_manager.split_data_strict(train, 0.8)
    
    #Save training data
    print("saving data...")
    X_train_path = os.path.join(PATH, "%s/datasets/train_prefixes.csv" % (dataset_ref))
    train.to_csv(X_train_path, index = False)
    X_test_path = os.path.join(PATH, "%s/datasets/test_prefixes.csv" % (dataset_ref))
    test.to_csv(X_test_path, index = False)
    X_val_path = os.path.join(PATH, "%s/datasets/val_prefixes.csv" % (dataset_ref))
    val.to_csv(X_val_path, index = False)

    

setting up data...
saving data...


In [5]:
len(dataset_manager.get_case_ids(val))

183